In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, matthews_corrcoef, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
import MachineLearning as ML
import PrepareData
import importlib

importlib.reload(ML)
importlib.reload(PrepareData)

<module 'PrepareData' from 'C:\\Users\\Adam\\source\\repos\\WeatherPrediction\\WeatherPredictionProject\\PrepareData.py'>

In [3]:
# method 'teach_models' takes the data and tech all models with it

def teach_models(X_train, y_train):
    models=[]

    X_train, y_train = SMOTETomek().fit_resample(X_train, y_train)

    models.append(ML.NeuralNetworks(X_train, y_train))
    models.append(ML.KNN(X_train, y_train))
    models.append(ML.DecisionTree(X_train, y_train))
    models.append(ML.LinearRegressionModel(X_train, y_train))
    models.append(ML.LogisticRegressionModel(X_train, y_train))
    models.append(ML.RandomForest(X_train, y_train))
    models.append(ML.SupportVectorMachines(X_train, y_train))
    models.append(ML.SupportVectorRegression(X_train, y_train))
    models.append(ML.TreeGradientBoosting(X_train, y_train))
    models.append(ML.DummyModel(X_train, y_train))

    return models

In [4]:
# method 'score_models' calculate scores to many statistics for all models separately

def score_models(models, X_test, y_test):
    results=[]
    for model in models:
        scores={}
        y_pred=model.predict(X_test)>0.5
        scores['Model']=model.__str__()
        scores['Accuracy']=accuracy_score(y_test, y_pred)
        scores['F1_score']=f1_score(y_test, y_pred)
        scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred)
        scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred)
        scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred))
        scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred)
        scores['Confusion_matrix']=confusion_matrix(y_test, y_pred)

        results.append(scores)
    return results

In [5]:
# method 'score_merged_models' calculate scores to many statistics for selected models as merged object

def score_merged_models(models, X_test, y_test):
    merged_scores={}
    results=pd.DataFrame()
    num=1
    for model in models:
        column="Model_"+str(num)
        results[column]=list(model.predict(X_test)>0.5)
        num+=1

    results=results.astype(np.int32)

    results["Sum"]=results.sum(axis=1)/len(models)>0.5

    y_pred_merged=results["Sum"].values>0.5

    merged_scores['Model']='Merged model'
    merged_scores['Accuracy']=accuracy_score(y_test, y_pred_merged)
    merged_scores['F1_score']=f1_score(y_test, y_pred_merged)
    merged_scores['Matthwes_correlation_coefficient_(MCC)']=matthews_corrcoef(y_test, y_pred_merged)
    merged_scores['Mean_squared_error_(MSE)']=mean_squared_error(y_test, y_pred_merged)
    merged_scores['Square_root_of_mean_squared_error_(RMSE)']=math.sqrt(mean_squared_error(y_test, y_pred_merged))
    merged_scores['Mean_absolute_error_(MAE)']=mean_absolute_error(y_test, y_pred_merged)
    merged_scores['Confusion_matrix']=confusion_matrix(y_test, y_pred_merged)

    return merged_scores

In [6]:
# cell with data read, split it into train and test parts and teach the models

df=PrepareData.get_data()

df=df.dropna()

y=df['nextday_rainfall']
X=df.drop('nextday_rainfall', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, stratify=y)

models=teach_models(X_train, y_train)

Epoch 1/25
329/329 [==============================] - 2s 4ms/step - loss: 0.6920 - f1: 0.4166 - accuracy: 0.5097 - val_loss: 0.6908 - val_f1: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6826 - f1: 0.4539 - accuracy: 0.5542 - val_loss: 0.6564 - val_f1: 0.6998 - val_accuracy: 0.6689
Epoch 3/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6618 - f1: 0.5751 - accuracy: 0.6131 - val_loss: 0.6374 - val_f1: 0.6378 - val_accuracy: 0.6712
Epoch 4/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6473 - f1: 0.5910 - accuracy: 0.6374 - val_loss: 0.6339 - val_f1: 0.6588 - val_accuracy: 0.6775
Epoch 5/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6501 - f1: 0.6008 - accuracy: 0.6353 - val_loss: 0.6210 - val_f1: 0.6923 - val_accuracy: 0.6809
Epoch 6/25
329/329 [==============================] - 1s 3ms/step - loss: 0.6440 - f1: 0.6127 - accuracy: 0.6353 - val_loss: 0.6263 - va

In [8]:
# simple usage of 'score_models' method

scores=score_models(models,X_test,y_test)
scores[0]

63/63 [==============================] - 0s 3ms/step


{'Model': '<keras.engine.sequential.Sequential object at 0x00000219BB391640>',
 'Accuracy': 0.701841712294674,
 'F1_score': 0.7369345630215195,
 'Matthwes_correlation_coefficient_(MCC)': 0.4082627850061699,
 'Mean_squared_error_(MSE)': 0.29815828770532604,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5460387236316908,
 'Mean_absolute_error_(MAE)': 0.29815828770532604,
 'Confusion_matrix': array([[571, 196],
        [403, 839]], dtype=int64)}

In [9]:
# simple usage of 'score_merged_models' method

scores=score_merged_models(models,X_test,y_test)
scores

63/63 [==============================] - 0s 2ms/step


{'Model': 'Merged model',
 'Accuracy': 0.7127924340467895,
 'F1_score': 0.7535241349850491,
 'Matthwes_correlation_coefficient_(MCC)': 0.41696062515953697,
 'Mean_squared_error_(MSE)': 0.28720756595321056,
 'Square_root_of_mean_squared_error_(RMSE)': 0.5359174992041318,
 'Mean_absolute_error_(MAE)': 0.28720756595321056,
 'Confusion_matrix': array([[550, 217],
        [360, 882]], dtype=int64)}

In [ ]:
# The below algorithm allow determining which set of models is best (needs ~10 min)

best_score=0
best_models=[]
num=0

import itertools
for L in range(2,len(models) + 1):
    for subset in itertools.combinations(models[:-1], L):
        merged_scores=score_merged_models(list(subset),X_test,y_test)
        if merged_scores['F1_score']>best_score:
            best_score=merged_scores['F1_score']
            best_models=list(subset)
        if num%100==0:
            print(num)
        num+=1

print(best_score)
print(best_models)